In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import glob
import matplotlib.pyplot as plt
import math
import cv2


%matplotlib inline

In [7]:
# class Features:
""" Images should be sent in RGB format """
    
def resize(img,size):
    """size is a tuple"""
    """ returns resized images"""
    return cv2.resize(img,size)

def to_hsv(img):
    """return image in HSV space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2HSV)

def to_gray(img):
    """return image in gray space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def sum(arr):
    """returns sum and no. of pixels between 20 and 240"""
    sum = 0
    count = 0
    for i in arr:
        for j in i:
            if(j>20 and j<240): #only pixels whose value is between 20 and 240
                sum+=j
                count+=1

    return (sum,count)

def pooling(image, pool_size, code, padding):
    """
    different codes for different pooling
    code min :min pooling
    code max :max pooling 
    code mean :mean pooling 
    code std :standard deviation pooling
    returns a image with padding operation and pooling operation
    """

    padded = arr = np.zeros((image.shape[0] + padding*2, 
                       image.shape[1] + padding*2))
    
    #  inserting image into zero array
    padded[int(padding):-int(padding), int(padding):-int(padding)] = image
    
    
    # print(f'original image size: {image.shape}')
    # print(f'padded image size: {padded.shape}')

    input_height, input_width = padded.shape
    pool_height, pool_width = pool_size

    # Calculate the output dimensions
    output_height = input_height - pool_height + 1
    output_width = input_width - pool_width + 1

    # Initialize the output data
    output_data = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            # Extract the region of interest (ROI)
            roi = padded[i : i + pool_height, j : j + pool_width]
            
            if code=='min':
                # Apply min pooling within the ROI
                output_data[i, j] = np.min(roi)

            if code=='max':
                # Apply max pooling within the ROI
                output_data[i, j] = np.max(roi)

            if code=='mean':
                # Apply mean pooling within the ROI
                output_data[i, j] = np.mean(roi)

            if code=='std':
                # Apply min pooling within the ROI
                output_data[i, j] = np.std(roi)


    # print(f'{code} pooled image size: {output_data.shape}')
    return output_data

def feature(data):
    """Return all the 12 features as a numpy array"""
    number,img,label = data
    img = resize(img,(250,250))

    #RGB SPACE
    r, g, b = cv2.split(img)
    sum_img = [sum(r),sum(g),sum(b),sum(r-g)]
    mean_features = [i[0]/i[1] for i in sum_img]
    mean_r,mean_g,mean_b,mean_rg = mean_features
    # 4 features done in RGB SPACE

    
    #HSV SPACE
    hsv = to_hsv(img)
    h,s,v = cv2.split(hsv)
    h = h/h.max()
    nH = np.count_nonzero(h>0.95)
    HHR = nH/h.size
    # HHR found

    
    #GRAY SPACE
    gray = to_gray(img)
    B_sum, B_size = sum(gray)
    B = B_sum/B_size # FOUND B

    #ENTROPY in gray space
    eq = cv2.equalizeHist(gray)
    unique, counts = np.unique(eq, return_counts=True)
    #only pixels whose value is between 20 and 240
    total_counts = counts[21:240].sum() 
    Ent = np.sum(np.array([-i*(i/total_counts)*math.log((i/total_counts),2) for i in counts[21:240]])) #Found Entropy

    #Calculating the 'G' features
    Ixy = gray
    min_Ixy = pooling(image=Ixy, pool_size=(3,3), code='min', padding=1)
    max_Ixy = pooling(image=Ixy, pool_size=(3,3), code='max', padding=1)
    mean_Ixy = pooling(image=Ixy, pool_size=(3,3), code='mean', padding=1)
    std_Ixy = pooling(image=Ixy, pool_size=(3,3), code='std', padding=1)
    
    g1 = Ixy - min_Ixy
    g2 = max_Ixy - Ixy
    g3 = Ixy - mean_Ixy
    g4 = std_Ixy
    g5 = Ixy
    
    G1 = g1.sum()/g1.size
    G2 = g2.sum()/g2.size
    G3 = g3.sum()/g3.size
    G4 = g4.sum()/g4.size
    G5 = g5.sum()/g5.size

    feature_all = [number,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label]
    return feature_all

In [8]:
folders = sorted(glob.glob("../India_95/complete/*"))
images = []
for folder in folders:
    all_images = []
    for i in os.listdir(folder):
        path = os.path.join(folder,i)
        all_images.append(path)
    images.append([i for i in all_images if "forniceal_palpebral" in i][0])
print(f"Total Number of Images = {len(images)}")

Total Number of Images = 95


In [9]:
def mask(filename):
    input_image = Image.open(filename)
    input_image.load()
    image = Image.new("RGB", input_image.size, (255, 255, 255))
    image.paste(input_image, mask = input_image.split()[3])
    return np.array(image)

def canny_edge_detection(frame): 
    # Convert the frame to grayscale for edge detection 
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) 
        
    # Apply Gaussian blur to reduce noise and smoothen edges 
    # blurred = cv2.GaussianBlur(src=gray, ksize=(3, 5), sigmaX=0.5)  #CHANGE gray ---> blurred

    # Perform Canny edge detection 
    edges = cv2.Canny(gray, 70, 135)
      
    return edges

def get_contours(image,edges):

    # # define a (3, 3) structuring element
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # # apply the dilation operation to the edged image
    # dilate = cv2.dilate(edges, kernel, iterations=1) #CHANGE edge --> dilate

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_contours=sorted(contours, key=cv2.contourArea, reverse= True)
    
    image_copy = image.copy()
    
    # draw the contours on a copy of the original image
    cv2.drawContours(image_copy, sorted_contours, -1, (0, 255, 0), 2) 
    # print(len(contours), "object was found in this image.")

    return image_copy,contours

def crop(image,contours):
    c = max(contours, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    # cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 1)

    img_copy = image.copy()
    cropped_img=img_copy[y:y+h, x:x+w]

    return cropped_img

In [11]:
df = pd.read_excel("../India_95/India.xlsx",0)

In [13]:
cropped_img_folder = []
for i in range(len(images)):

    img = mask(images[i])
    edges = canny_edge_detection(img)
    img_copy,contours = get_contours(img,edges)
    cropped_img = crop(img,contours)

    number = int(images[i].split("/")[-2]) 
    
    label = df.loc[df['Number'] == int(number)]['Hgb'].tolist()[0]
    
    data = [number,cropped_img,label]
    
    cropped_img_folder.append(data)

In [14]:
All_Data = []

for i in cropped_img_folder:
    All_Data.append(feature(i))

print(len(All_Data))

95


In [15]:
cols = ['number','mean_r','mean_g','mean_b','mean_rg','HHR','Ent','B','G1','G2','G3','G4','G5','label']

complete_data = pd.DataFrame(All_Data,columns=cols)

In [17]:
complete_data.head()

,number,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label
0,1,166.115510,84.085488,118.026636,82.950664,0.200800,16530.859714,112.137698,8.106592,4.140496,1.349161,4.895221,191.313664,12.2
1,10,160.143607,88.695460,120.634118,72.781835,0.235520,15663.471752,113.683637,8.385424,4.904768,1.255196,5.044717,178.182672,11.3
2,11,167.294504,106.190649,144.654703,61.616165,0.080352,16258.330210,128.833414,8.423712,4.667440,1.271712,4.985541,192.234128,13.2
3,12,175.469784,124.728785,156.774717,51.857562,0.096384,16015.872101,143.585054,7.037088,3.561568,1.227013,4.158212,210.090256,10.6
4,13,174.038383,113.084957,149.969145,61.577065,0.077600,16105.892531,135.513296,7.608624,3.746800,1.337888,4.442796,189.553616,10.6


In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [18]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(complete_data, test_size=0.3)

In [19]:
# Separate features and target variable in train data
X_train = train_data.drop(columns=['number','label'])
y_train = train_data['label']

# Separate features and target variable in test data
X_test = test_data.drop(columns=['number','label'])
y_test = test_data['label']

In [20]:
algorithms = {
    'Linear Regression': LinearRegression(),
    'SVM Regression': SVR(kernel='poly'),  # Adjust kernel as needed
    'RandomForest': RandomForestRegressor(),
    'Gradient Boost': GradientBoostingRegressor(),
    'knn': KNeighborsRegressor(),
    'LGBM': LGBMRegressor(),
    'CatBoost': CatBoostRegressor(),
    'Kernel Ridge Regressor': KernelRidge(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'XG Boost': XGBRegressor()
}

In [21]:
# Metric tables
metric_table_train = pd.DataFrame()
metric_table_test = pd.DataFrame()

In [22]:
# Run the algorithms ... create metrics and plots
for algorithm_name, algorithm in algorithms.items():

    # Train model
    algorithm.fit(X_train, y_train)

    # Train predictions
    y_train_pred = algorithm.predict(X_train)

    # Test predictions
    y_test_pred = algorithm.predict(X_test)

    # Train metrics
    r2_train = algorithm.score(X_train, y_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    # Test metrics
    r2_test = algorithm.score(X_test, y_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    # Additional metrics using statsmodels for all algorithms
    residuals_train = y_train - y_train_pred
    residuals_test = y_test - y_test_pred

    durbin_watson_stat_train = sm.stats.durbin_watson(residuals_train)
    jb_stat_train, jb_p_value_train, _, _ = sm.stats.jarque_bera(residuals_train)

    durbin_watson_stat_test = sm.stats.durbin_watson(residuals_test)
    jb_stat_test, jb_p_value_test, _, _ = sm.stats.jarque_bera(residuals_test)

    # Update metric tables
    metric_table_train.at[algorithm_name, 'MAE'] = mae_train
    metric_table_train.at[algorithm_name, 'R-squared'] = r2_train
    metric_table_train.at[algorithm_name, 'MSE'] = mse_train
    metric_table_train.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_train
    metric_table_train.at[algorithm_name, 'Jarque-Bera'] = jb_stat_train
    metric_table_train.at[algorithm_name, 'JB P-value'] = jb_p_value_train

    metric_table_test.at[algorithm_name, 'MAE'] = mae_test
    metric_table_test.at[algorithm_name, 'R-squared'] = r2_test
    metric_table_test.at[algorithm_name, 'MSE'] = mse_test
    metric_table_test.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_test
    metric_table_test.at[algorithm_name, 'Jarque-Bera'] = jb_stat_test
    metric_table_test.at[algorithm_name, 'JB P-value'] = jb_p_value_test


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001184 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 276
[LightGBM] [Info] Number of data points in the train set: 66, number of used features: 12
[LightGBM] [Info] Start training from score 11.193939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [23]:
# Display metrics in tables
print("Metrics - Train Data:\n")
print(metric_table_train.to_string())
print("-------------------------------------------------")

print("Metrics - Test Data:\n")
print(metric_table_test.to_string())

Metrics - Train Data:

                             MAE  R-squared           MSE  Durbin-Watson  Jarque-Bera  JB P-value
Linear Regression       1.063210   0.524669  1.993781e+00       1.805952     3.304594    0.191609
SVM Regression          1.659379   0.006534  4.167102e+00       2.160462     1.546792    0.461443
RandomForest            0.518348   0.902557  4.087245e-01       2.046413     1.520581    0.467530
Gradient Boost          0.097018   0.996643  1.408186e-02       1.685949     0.556744    0.757015
knn                     1.535455   0.195645  3.373873e+00       2.005720     0.549904    0.759609
LGBM                    1.019968   0.591431  1.713745e+00       2.165197     1.670418    0.433784
CatBoost                0.019508   0.999861  5.812096e-04       2.079517     2.621409    0.269630
Kernel Ridge Regressor  1.191355   0.457044  2.277433e+00       1.801973     2.534097    0.281662
Elastic Net             1.335692   0.350977  2.722334e+00       2.007490     2.512784    0.2846

In [24]:
import pycaret

In [26]:
from pycaret.regression import *
s = setup(complete_data, target='label', ignore_features=['number'], preprocess=False, session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(95, 14)"
4,Transformed data shape,"(95, 13)"
5,Transformed train set shape,"(66, 13)"
6,Transformed test set shape,"(29, 13)"
7,Ignore features,1
8,Numeric features,12


In [27]:
best_r = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.3557,2.9166,1.6097,0.2896,0.1328,0.1249,0.2940
et,Extra Trees Regressor,1.3130,2.8634,1.6207,0.2094,0.1352,0.1226,0.0100
ada,AdaBoost Regressor,1.2650,2.5954,1.5529,0.1831,0.1274,0.1166,0.0070
rf,Random Forest Regressor,1.3223,2.6930,1.5742,0.1696,0.1302,0.1224,0.0120
gbr,Gradient Boosting Regressor,1.3952,2.9293,1.6416,0.1427,0.1375,0.1287,0.0070
ridge,Ridge Regression,1.3623,3.0392,1.6702,0.1062,0.1361,0.1240,0.1040
huber,Huber Regressor,1.4067,3.1594,1.7074,0.0997,0.1391,0.1279,0.0030
lasso,Lasso Regression,1.4120,3.1648,1.7026,0.0995,0.1400,0.1288,0.1150
llar,Lasso Least Angle Regression,1.4120,3.1650,1.7027,0.0995,0.1400,0.1288,0.0070
lightgbm,Light Gradient Boosting Machine,1.4247,3.2860,1.7117,0.0842,0.1402,0.1296,0.0070


In [28]:
print(best_r)

In [29]:
evaluate_model(best_r)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [30]:
predict_model(best_r)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,1.1598,2.1582,1.4691,0.4291,0.1159,0.1009


,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label,prediction_label
71,175.925720,126.581451,154.250595,50.055267,0.190272,16804.599609,144.484924,7.679888,3.663984,1.346695,4.461142,200.818848,9.1,9.730712
62,138.608643,77.041153,115.367226,62.658043,0.109568,13782.414062,99.836166,8.414496,4.539408,1.326535,5.021771,203.675323,13.8,12.013835
29,158.704285,86.480194,126.543617,72.431419,0.270784,11816.129883,112.631630,7.364720,4.027808,1.136500,4.335993,153.293381,9.7,11.319360
53,167.599197,95.245903,128.758453,72.956291,0.247920,16235.455078,120.707909,7.797136,3.970320,1.341445,4.696663,195.005295,12.5,12.395563
90,168.967224,92.357147,130.296432,77.023102,0.238656,16160.344727,119.596275,7.965248,3.984880,1.349547,4.707871,188.017883,13.4,12.405707
4,174.038376,113.084953,149.969147,61.577065,0.077600,16105.892578,135.513290,7.608624,3.746800,1.337888,4.442797,189.553619,10.6,11.085646
31,169.107239,98.704597,135.013550,70.810814,0.147504,15583.392578,123.901398,7.465680,3.809648,1.253673,4.413886,204.016464,11.1,11.679743
77,162.976715,75.856606,105.529968,88.623070,0.320928,14526.099609,104.785522,8.321152,5.164864,1.130252,4.980634,160.791092,13.0,12.728972
79,153.740143,86.112801,121.020012,68.748116,0.137424,15083.416992,110.341164,8.306480,4.422640,1.350149,5.048759,201.483841,11.0,11.984407
70,177.905777,123.975029,161.672729,54.421707,0.077872,16069.750000,144.409607,7.497120,3.580656,1.342270,4.405998,202.220474,12.3,9.711666


In [31]:
tuned_catboost = tune_model(best_r, fold = 10, n_iter = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.6608,0.8368,0.9148,-0.0470,0.0739,0.0607
1,1.0392,1.3260,1.1515,0.5269,0.1035,0.1066
2,1.3840,3.9871,1.9968,0.0305,0.1863,0.1602
3,1.5990,3.7668,1.9408,0.5680,0.1576,0.1398
4,1.2905,2.3188,1.5228,0.4403,0.1339,0.1266
5,0.8231,1.1801,1.0863,0.5836,0.0983,0.0817
6,0.8861,1.2887,1.1352,0.2976,0.0862,0.0705
7,1.8433,3.7446,1.9351,0.4376,0.1555,0.1584
8,0.9761,1.2521,1.1190,0.6469,0.0807,0.0767


Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [32]:
evaluate_model(tuned_catboost)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [33]:
predict_model(tuned_catboost)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,1.1978,2.2488,1.4996,0.4051,0.1181,0.1044


,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label,prediction_label
71,175.925720,126.581451,154.250595,50.055267,0.190272,16804.599609,144.484924,7.679888,3.663984,1.346695,4.461142,200.818848,9.1,10.006741
62,138.608643,77.041153,115.367226,62.658043,0.109568,13782.414062,99.836166,8.414496,4.539408,1.326535,5.021771,203.675323,13.8,11.671827
29,158.704285,86.480194,126.543617,72.431419,0.270784,11816.129883,112.631630,7.364720,4.027808,1.136500,4.335993,153.293381,9.7,10.893827
53,167.599197,95.245903,128.758453,72.956291,0.247920,16235.455078,120.707909,7.797136,3.970320,1.341445,4.696663,195.005295,12.5,12.539658
90,168.967224,92.357147,130.296432,77.023102,0.238656,16160.344727,119.596275,7.965248,3.984880,1.349547,4.707871,188.017883,13.4,12.487544
4,174.038376,113.084953,149.969147,61.577065,0.077600,16105.892578,135.513290,7.608624,3.746800,1.337888,4.442797,189.553619,10.6,10.800099
31,169.107239,98.704597,135.013550,70.810814,0.147504,15583.392578,123.901398,7.465680,3.809648,1.253673,4.413886,204.016464,11.1,11.250892
77,162.976715,75.856606,105.529968,88.623070,0.320928,14526.099609,104.785522,8.321152,5.164864,1.130252,4.980634,160.791092,13.0,13.522781
79,153.740143,86.112801,121.020012,68.748116,0.137424,15083.416992,110.341164,8.306480,4.422640,1.350149,5.048759,201.483841,11.0,11.687661
70,177.905777,123.975029,161.672729,54.421707,0.077872,16069.750000,144.409607,7.497120,3.580656,1.342270,4.405998,202.220474,12.3,9.783463


### Regression to Classification

--> <11.0 --> Anemic

--> >=11.0 ---> Non-Anemic 

In [34]:
tuned_predict = predict_model(tuned_catboost)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,1.1978,2.2488,1.4996,0.4051,0.1181,0.1044


In [35]:
tuned_predict.columns.values

array(['mean_r', 'mean_g', 'mean_b', 'mean_rg', 'HHR', 'Ent', 'B', 'G1',
       'G2', 'G3', 'G4', 'G5', 'label', 'prediction_label'], dtype=object)

In [36]:
actual_tuned = (tuned_predict['label']<11.0).tolist()
predicted_tuned = (tuned_predict['prediction_label']<11.0).tolist()

In [37]:
TP=TN=FN=FP = 0
for i in range(len(actual_tuned)):
    if(actual_tuned[i]==True and predicted_tuned[i]==True):
        TP +=1
    if(actual_tuned[i]==False and predicted_tuned[i]==False):
        TN +=1
    if(actual_tuned[i]==True and predicted_tuned[i]==False):
        FN +=1
    if(actual_tuned[i]==False and predicted_tuned[i]==True):
        FP +=1

print(f"Accuracy = {(TP+TN)/(TP+TN+FP+FN)}")
print(f"Precision = {(TP)/(TP+FP)}")
print(f"Sensitivity = {(TP)/(TP+FN)}")
print(f"Specificity = {(TN)/(TN+FP)}")

Accuracy = 0.8275862068965517
Precision = 0.7272727272727273
Sensitivity = 0.8
Specificity = 0.8421052631578947
